In [155]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image_dataset_from_directory
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [156]:
testing = image_dataset_from_directory("Brain_tumor\\Testing", pad_to_aspect_ratio=True)
training = image_dataset_from_directory("Brain_tumor\\Training", pad_to_aspect_ratio=True)

Found 1311 files belonging to 4 classes.
Found 5712 files belonging to 4 classes.


In [157]:
training_set = training.take(int(0.25*len(training)))
testing_set = testing.take(int(0.25*len(testing)))

In [158]:
def data_vectorizer(data:tf.data.Dataset):
    data = data.unbatch()
    size = len(list(data))
    vector = np.empty((size, 256*256*3), dtype=np.float32)
    labels = np.empty((size,))
    j = 0
    for img, label in data:
        vector[j, :] = img.numpy().flatten()
        labels[j] = label.numpy()
        j += 1
    return vector, labels

In [159]:
x_train, y_train = data_vectorizer(training_set)
x_test, y_test = data_vectorizer(testing_set)

In [160]:
print("Training set size: {}".format(np.shape(x_train)))
print("Testing set size: {}".format(np.shape(x_test)))

Training set size: (1408, 196608)
Testing set size: (320, 196608)


In [161]:
scaler = MinMaxScaler((0,1)).fit(x_train, y_train)
scaled_x_train = scaler.transform(x_train)
percep = Perceptron().fit(scaled_x_train, y_train)
model = OneVsRestClassifier(percep).fit(scaled_x_train, y_train)

In [172]:
from sklearn.metrics import accuracy_score
outputs = model.predict(scaler.transform(x_test))
print("Accuracy: {}".format(accuracy_score(y_test, outputs)))
print("Precision: {}".format(precision_score(y_test, outputs, average='weighted')))
print("Recall: {}".format(recall_score(y_test, outputs, average='weighted')))
print("F1: {}".format(f1_score(y_test, outputs, average='weighted')))

Accuracy: 0.775
Precision: 0.7737355860350263
Recall: 0.775
F1: 0.7726091619236293
